### 모델 학습하기

1. 저장된 데이터를 읽어와서 신경망 모델에 입력할 수 있는 형태로 변경
1. 입력조건이나 네트워크 크기 및 학습 알고리즘을 변경하면서 학습효과가 높은 조건 확인
  - **인공지능 분야에서 얘기하는 노가다 작업**
1. 만족할 만한 조건을 확인하면 해당 조건으로 학습된 모델을 저장
1. 실전에서는 저장된 모델을 불러와서 사용하면 됨.

In [1]:
import pickle
import numpy as np
from tensorflow import keras

POKER_CARD_LENGTH = 52

In [2]:
def load_image_data(type_='train'):
    if type_ not in ('train', 'test'):
        raise Exception(f"type error(train, test); {type_}")

    return pickle.load(open(f"../{type_}.pickle", 'rb'))


def load_train_data():
    train_ = load_image_data('train')
    return np.array([r[0] for r in train_]), np.array([r[1] for r in train_])


def load_test_data():
    test_ = load_image_data('test')
    return np.array([r[0] for r in test_]), np.array([r[1] for r in test_])

In [3]:
x_train, y_train = load_train_data()
x_test, y_test = load_test_data()

# one hot encoding
y_train = keras.utils.to_categorical(y_train, POKER_CARD_LENGTH)
y_test = keras.utils.to_categorical(y_test, POKER_CARD_LENGTH)

In [4]:
x_train = x_train.reshape([x_train.shape[0], x_train.shape[1] * x_train.shape[2]])
x_test = x_test.reshape([x_test.shape[0], x_test.shape[1] * x_test.shape[2]])

In [5]:
print(x_train.shape, x_test.shape)

(1924, 1800) (624, 1800)


In [6]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

### 이후부터는 네트워크 모델을 바꿔가면서 반복적인 테스트가 필요함.

In [13]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, activation="relu", input_shape=(x_train.shape[1],)))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(POKER_CARD_LENGTH, activation="softmax"))

In [14]:
optimizer = keras.optimizers.SGD(lr=0.01)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               230528    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 52)                6708      
Total params: 253,748
Trainable params: 253,748
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x_train, y_train,
          batch_size=4,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 1924 samples, validate on 624 samples
Epoch 1/10
1924/1924 [==============================] - 1s 536us/sample - loss: 3.8582 - acc: 0.0567 - val_loss: 3.5793 - val_acc: 0.1923
Epoch 2/10
1924/1924 [==============================] - 1s 478us/sample - loss: 3.0779 - acc: 0.2568 - val_loss: 2.3819 - val_acc: 0.4022
Epoch 3/10
1924/1924 [==============================] - 1s 441us/sample - loss: 1.7869 - acc: 0.5504 - val_loss: 1.2704 - val_acc: 0.6923
Epoch 4/10
1924/1924 [==============================] - 1s 438us/sample - loss: 0.8914 - acc: 0.8119 - val_loss: 0.6265 - val_acc: 0.8990
Epoch 5/10
1924/1924 [==============================] - 1s 433us/sample - loss: 0.4212 - acc: 0.9361 - val_loss: 0.3242 - val_acc: 0.9551
Epoch 6/10
1924/1924 [==============================] - 1s 431us/sample - loss: 0.2201 - acc: 0.9818 - val_loss: 0.1834 - val_acc: 0.9968
Epoch 7/10
1924/1924 [==============================] - 1s 436us/sample - loss: 0.1267 - acc: 0.9953 - val_loss: 0.1095 - val

In [16]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.04513678780924051
Test accuracy: 1.0


#### sigmoid 에서는 학습되지 않고 relu, softmax 적용시 학습되는 것을 확인함.

In [22]:
from tensorflow.python.keras.layers import Dropout

In [27]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, activation="relu", input_shape=(x_train.shape[1],)))
model.add(keras.layers.Dense(128, activation="relu"))
#model.add(Dropout(0.5))
model.add(keras.layers.Dense(POKER_CARD_LENGTH, activation="softmax"))

In [28]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 128)               230528    
_________________________________________________________________
dense_17 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_18 (Dense)             (None, 52)                6708      
Total params: 253,748
Trainable params: 253,748
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(x_train, y_train,
          batch_size=4,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 1924 samples, validate on 624 samples
Epoch 1/10
1924/1924 [==============================] - 2s 794us/sample - loss: 2.8976 - acc: 0.2510 - val_loss: 1.3322 - val_acc: 0.6362
Epoch 2/10
1924/1924 [==============================] - 1s 668us/sample - loss: 0.6370 - acc: 0.8170 - val_loss: 0.3699 - val_acc: 0.8878
Epoch 3/10
1924/1924 [==============================] - 1s 646us/sample - loss: 0.2079 - acc: 0.9454 - val_loss: 0.1430 - val_acc: 0.9760
Epoch 4/10
1924/1924 [==============================] - 1s 626us/sample - loss: 0.0940 - acc: 0.9771 - val_loss: 0.1702 - val_acc: 0.9583
Epoch 5/10
1924/1924 [==============================] - 1s 617us/sample - loss: 0.1304 - acc: 0.9579 - val_loss: 0.2854 - val_acc: 0.9151
Epoch 6/10
1924/1924 [==============================] - 1s 623us/sample - loss: 0.1517 - acc: 0.9553 - val_loss: 0.0998 - val_acc: 0.9631
Epoch 7/10
1924/1924 [==============================] - 1s 647us/sample - loss: 0.0274 - acc: 0.9917 - val_loss: 0.0025 - val

In [30]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0008105412480057193
Test accuracy: 1.0


### 저장하기 및 불러오기

In [31]:
# model.save('../model_poker_card.h5')

In [32]:
from tensorflow.python.keras.models import load_model

In [34]:
model2 = load_model('../model_poker_card.h5')

In [35]:
xhat_idx = np.random.choice(x_test.shape[0], 5)
xhat = x_test[xhat_idx]

In [36]:
yhat = model.predict_classes(xhat)

In [38]:
from numpy import argmax

In [39]:
for i in range(5):
    print('True : ' + str(argmax(y_test[xhat_idx[i]])) + ', Predict : ' + str(yhat[i]))

True : 16, Predict : 16
True : 22, Predict : 22
True : 30, Predict : 30
True : 8, Predict : 8
True : 37, Predict : 37


#### model.predict 와 model.predict_classes 의 차이를 확인하기 위해 실행해봄.

In [40]:
yhat = model.predict(xhat)

In [43]:
for i in range(5):
    print('True : ' + str(argmax(y_test[xhat_idx[i]])) + ', Predict : ' + str(yhat[i]))

True : 16, Predict : [3.35516347e-18 2.22143779e-18 5.67638509e-18 8.70893302e-07
 6.38144614e-19 5.83728573e-16 2.25347313e-15 9.10555946e-14
 2.83132469e-14 4.07546677e-18 4.15725699e-19 2.13979281e-19
 3.33026954e-20 6.25090502e-20 1.35447625e-11 7.21071980e-10
 9.99530315e-01 2.65343338e-11 6.19699714e-09 1.39422411e-07
 1.13040342e-05 4.34204878e-04 6.66711152e-13 1.75020482e-17
 1.98517081e-18 6.72894263e-21 5.74596252e-13 6.19452458e-21
 6.34423902e-11 3.24259174e-13 5.26069693e-13 1.13086776e-15
 1.52504265e-09 1.38143715e-13 7.18817930e-08 2.92825185e-14
 1.25873509e-12 4.34348225e-16 2.05994029e-18 7.87720312e-13
 9.07292248e-16 2.62409507e-08 3.38581455e-11 2.75765188e-10
 8.82073303e-10 9.65684990e-07 3.28690417e-12 2.21875489e-05
 1.84974345e-11 3.66516217e-09 2.13460716e-12 1.28369456e-18]
True : 22, Predict : [3.66905129e-09 1.01217667e-12 1.17433962e-13 1.22764004e-06
 1.42734899e-08 4.71570416e-07 4.04350707e-13 1.15488829e-05
 1.63792919e-07 1.03686602e-04 4.85706510e

In [44]:
for i in range(5):
    print('True : ' + str(argmax(y_test[xhat_idx[i]])) + ', Predict : ' + str(argmax(yhat[i])))

True : 16, Predict : 16
True : 22, Predict : 22
True : 30, Predict : 30
True : 8, Predict : 8
True : 37, Predict : 37
